In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import pickle
from pathlib import Path
import numpy as np
import random
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from import_casa import casa, cano
from casa.annot import AspectEnum

In [7]:
random.seed(6523)

In [8]:
data_path = Path("../../../data/caprice/caprice_seq_data_20210430.pkl")
with data_path.open("rb") as fin:
    data = pickle.load(fin)
random.shuffle(data)

In [11]:
model_path = casa.get_data_path() / "caprice/seq-model-ep9"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [16]:
txt = "測試自串"
batch = tokenizer([txt], return_tensors="pt", padding=True)

In [92]:
dial = 10.
out = model(**batch, output_hidden_states=True)

In [93]:
print(out.logits)
out.logits[0,1].backward()

tensor([[ 3.1572, -1.6533, -2.0265]], grad_fn=<AddmmBackward>)


In [95]:
out.hidden_states[0].shape

torch.Size([1, 6, 768])

In [106]:
cls_hidden = np.dstack([x[0, 0, :].detach().numpy() for x in out.hidden_states]).squeeze().T

In [109]:
cls_hidden.shape

(13, 768)

In [124]:
from sklearn.decomposition import DictionaryLearning, NMF

In [123]:
3000*768*12*8/1024/1024

210.9375

In [125]:
nmf = NMF(n_components=10)
nmf_W = nmf.fit_transform(cls_hidden)

ValueError: Negative values in data passed to NMF (input X)

In [121]:
dict_learn = DictionaryLearning(n_components=10)
code = dict_learn.fit_transform(cls_hidden)

c:\python38\lib\site-packages\sklearn\decomposition\_dict_learning.py:174: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  new_code = orthogonal_mp_gram(


In [122]:
code

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.36585426e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.11160717e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.52054195e+01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-2.24791050e-01, -1.89658236e-02, -4.27643508e-02,
         5.57002909e-02,  1.77728847e-01, -2.21914887e-01,
         1.62942848e+01, -5.20834029e-01, -1.60758123e-02,
         1.19345132e-02],
       [ 6.33432925e-01,  4.91444282e-02,  1.05040967e-01,
        -1.41664907e-01, -4.51044440e-01,  5.58365703e-01,
         6.24289274e+00,  9.76596355e+00,  4.17317450e-02,
        -2.

In [86]:
output_.grad

tensor([[[ 2.0405e-03, -4.1365e-03,  1.3692e-03,  ...,  1.8974e-03,
          -3.4728e-04,  3.0908e-04],
         [ 4.6576e-04,  3.2694e-03, -2.6202e-03,  ...,  1.5930e-03,
           2.4502e-03,  5.5137e-03],
         [ 1.9376e-03,  2.8792e-03,  1.2040e-03,  ..., -5.3440e-04,
          -1.1346e-03,  3.0696e-03],
         [-2.4427e-03, -4.8167e-03,  5.7168e-03,  ..., -3.1514e-03,
          -2.7876e-03,  5.2438e-03],
         [-1.1823e-03, -3.8804e-03,  8.1164e-04,  ...,  1.3053e-03,
          -2.1243e-05, -7.6663e-03],
         [-8.8982e-04, -6.6297e-04,  6.3790e-04,  ..., -4.3090e-04,
          -2.2132e-03,  1.3988e-03]]])

In [87]:
output_.shape

torch.Size([1, 6, 3072])

In [81]:
input_ = None
output_ = None

def im_forward_hook(mod, input_state, output_state):
    global input_, output_
    output_state[0,0,:] *= dial
    output_state.retain_grad()
    input_ = input_state
    output_ = output_state

In [82]:
handle.remove()
handle = bert.encoder.layer[0].intermediate.register_forward_hook(im_forward_hook)

In [44]:
bert.encoder.layer[0].intermediate

BertIntermediate(
  (dense): Linear(in_features=768, out_features=3072, bias=True)
)

In [35]:
bert.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): BertLayer(
    (attention): BertAttention(
      (self)

In [33]:
3072/768

4.0